In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as scipy

In [3]:
def gaussian_kernel(X, sigma):
    distances = np.sum((X[:,np.newaxis] - X)**2, axis=-1)
    kernel_matrix = np.exp(-distances / (2*sigma**2))
    
    return kernel_matrix

In [2]:
def lasso_timed_lag(expression_data, time_data, target_gene, lags, lambda_val, sigma):
    
    #INPUTS:
    #expression_data: NxM matrix of expression data of N genes and M cells, ordered temporally with index matching time_data
    #time_data: NxM matrix of time points each expression data corresponds to index-wise
    #target_gene: integer between 0 and N-1 that represents the target gene's index in the expression_data matrix
    #lags: a list of integers of time lags to be included in the Am design matrix. Example: [5,10,50]
    #lambda_val: lambda value for lasso regression, adjusted based on data fit results
    #sigma: kernel smoothing parameter, set by user
    
    #OUTPUTS:
    #bm: nNx1 matrix of predicted coefficients, where n is the number of lags (i.e. len(lags)) and N genes
    
    
    X_design = np.asarray(expression_data)
    time_index = np.asarray(time_data)
    
    n_lags = len(lags)
    max_lag = max(lags)
    N = X_design.shape[0]
    M = X_design.shape[1]
    
    #find index of largest time lag (time>50, for instance)
    max_time_index=0
    for i in range(M):
        if time_index[target_gene, i]>max_lag:
            max_time_index=i #column number of the index of where maximum lag occurs
            break
        else:
            continue
    
    y = X_design[target_gene, max_time_index:] #dim: 1x(M-max_time_index+1)
    y = y.transpose() #dim: (M-max_time_index+1)x1
    
    #find kernel matrix for X design matrix
    X_kernel = gaussian_kernel(X_design, sigma)
    
    #create Am matrix, (essentially the X matrix in lasso regression)
    Am = np.empty(shape=[M-max_time_index+1, n_lags*N])
    for i in range(n_lags):
        #find the non-smoothed version first
        Am[:,i*N:(i+1)*N] = X_design[:,max_time_index-lags[i]:M-lags[i]].transpose()
        
        #apply (NOT YET) normalized kernal matrix to Am
        X_kernel_sec = X_kernel[:,max_time_index-lags[i]:M-lags[i]]
        Am_kernelized[:,i*N:(i+1)*N] = np.dot(Am[:,i*N:(i+1)*N], X_kernel_sec)
    
    
    
    #run glmnet regression
    bm = glmnet_lasso(y, Am_kernelized, 0.1)
    
    
    return bm